In [13]:
#Importo librerías necesarias
import json
import numpy as np
import ast
import re
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
from textblob import TextBlob

In [2]:
#Extraccion del json
row = [] #lista vacia para ir guardando las filas

with open ("australian_user_reviews.json", 'r', encoding='utf-8') as file: #utilizo with para que el archivo se abra y cierre
    for line in file.readlines(): #bucle para ir leyendo filas y luego agregarlas a row
        row.append(ast.literal_eval(line)) # interpreta las lineas del json y transforma en objeto de python

# genero el dataframe 
df_reviews = pd.DataFrame(row)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


NULOS

In [4]:
df_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

In [5]:
# Transoformación de la columna reviews: desanidado
# Creo lista vacía
desanidado_reviews = []

# Itero en cada fila
for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']
    
    # Itero en cada review
    for review in reviews:
        new_review = {
            'user_id': user_id,
            'user_url': user_url,
            'funny': review.get('funny', ''),
            'posted': review.get('posted', ''),
            'last_edited': review.get('last_edited', ''),
            'item_id': review.get('item_id', ''),
            'helpful': review.get('helpful', ''),
            'recommend': review.get('recommend', ''),
            'review_text': review.get('review', '')  # Renamed 'review' to 'review_text'
        }
        
        # Agrego la review a la lista
        desanidado_reviews.append(new_review)

# Create a new DataFrame 'df_reviews_unnested' from the list of unnested reviews
df_desanidado_reviews = pd.DataFrame(desanidado_reviews)

In [6]:
df_desanidado_reviews.head()


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [7]:
df_desanidado_reviews.isnull().sum()

user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review_text    0
dtype: int64

In [8]:
df_desanidado_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review_text  59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [9]:
#Duplicados
duplicados = df_desanidado_reviews[df_desanidado_reviews.duplicated(keep=False)]
duplicados

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text
31,76561198156664158,http://steamcommunity.com/profiles/76561198156...,,Posted June 16.,,252950,0 of 1 people (0%) found this review helpful,True,love it
115,Rivtex,http://steamcommunity.com/id/Rivtex,,"Posted December 23, 2015.",,298630,0 of 1 people (0%) found this review helpful,True,Its so bosss
278,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,,"Posted August 29, 2015.",,346110,0 of 1 people (0%) found this review helpful,True,"It's a good game, mechanics seem solid and the..."
279,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,,"Posted October 5, 2015.","Last edited October 5, 2015.",380770,0 of 5 people (0%) found this review helpful,False,"Well, I don't have much to say about the game...."
280,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,,Posted June 29.,,454890,No ratings yet,True,"It's a pretty good game, not bad for the 49 ce..."
...,...,...,...,...,...,...,...,...,...
44433,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...
44434,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,..."
44435,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien..."
44436,76561198092022514,http://steamcommunity.com/profiles/76561198092...,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...


In [10]:
#Analizo los duplicados: 
duplicados[duplicados["user_id"] == "Rivtex"]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text
115,Rivtex,http://steamcommunity.com/id/Rivtex,,"Posted December 23, 2015.",,298630,0 of 1 people (0%) found this review helpful,True,Its so bosss
7277,Rivtex,http://steamcommunity.com/id/Rivtex,,"Posted December 23, 2015.",,298630,0 of 1 people (0%) found this review helpful,True,Its so bosss


In [11]:
#Se eliminan 
df_desanidado_reviews = df_desanidado_reviews.drop_duplicates(keep='first')

In [12]:
df_desanidado_reviews.head(20)


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
5,js41637,http://steamcommunity.com/id/js41637,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...
6,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
7,evcentric,http://steamcommunity.com/id/evcentric,,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?"""
8,evcentric,http://steamcommunity.com/id/evcentric,,"Posted November 3, 2014.",,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ..."
9,evcentric,http://steamcommunity.com/id/evcentric,,"Posted October 15, 2014.",,263360,No ratings yet,True,"Random drops and random quests, with stat poin..."


In [14]:
#Creo funcion de analisis de sentimiento

""""
    -review (str): Texto de la review a analizar
    
    Return:
    -numero (int): Valor que se le cargo en la funcion de polaridad
                - 0 Negativo o Malo
                - 1 Neutral
                - 2 Positivo
"""
def analizar_sentimiento(review):
    if review is None:
        return 1 #Neutral
    analisis = TextBlob(review)
    polaridad = analisis.sentiment.polarity
    if polaridad > 0.1:
        return 2 #'Positivo'
    elif polaridad < -0.1:
        return 0 #'Negativo'
    else:
        return 1 #'Neutral'

In [15]:
#Aplico la función
df_desanidado_reviews["sentiment_analisis"]= df_desanidado_reviews["review_text"].astype(str).apply(analizar_sentimiento)
df_desanidado_reviews.head(25)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review_text,sentiment_analisis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1
5,js41637,http://steamcommunity.com/id/js41637,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,1
6,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
7,evcentric,http://steamcommunity.com/id/evcentric,,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?""",2
8,evcentric,http://steamcommunity.com/id/evcentric,,"Posted November 3, 2014.",,237930,No ratings yet,True,"Elegant integration of gameplay, story, world ...",2
9,evcentric,http://steamcommunity.com/id/evcentric,,"Posted October 15, 2014.",,263360,No ratings yet,True,"Random drops and random quests, with stat poin...",0


In [16]:
#One-Hot Enconding
df_desanidado_reviews['recommend'] = df_desanidado_reviews.recommend.map({True : 1, False : 0})

In [17]:
# Elimnar columnas
df_desanidado_reviews.drop(['user_url', 
                     'funny', 
                     'last_edited', 
                     'helpful', 
                     'review_text' ], axis=1, inplace=True)

In [18]:
df_desanidado_reviews.isnull().sum()

user_id               0
posted                0
item_id               0
recommend             0
sentiment_analisis    0
dtype: int64

In [19]:
# Transormar posted a año
df_desanidado_reviews['posted_year'] = df_desanidado_reviews['posted'].str.extract(r'(\d{4})')

#Eliminar posted
df_desanidado_reviews.drop(columns='posted', inplace=True)

In [21]:
#Guardado PARQUET
df_desanidado_reviews.to_parquet('Data/reviews.parquet', engine='pyarrow')

#df_reviews_unnested.to_parquet('../data/fixed/reviews.parquet')

In [20]:
#Guardado CSV Opcional

#df_desanidado_reviews.to_csv('Data/reviews.csv', index=False, encoding="utf-8")